In [1]:
# Этап 0. Импорт библиотек и модулей проекта.
from torch.utils.data import DataLoader
from src.data_loader import train_dataset, val_dataset, classes, len_classes

from torchvision.models import resnet18, ResNet18_Weights
import torch
import torch.nn as nn
from torch.optim import Adam

from src.training import training
from sklearn.metrics import classification_report

In [2]:
# Этап 1. Подготовка данных.

# Размер батча
batch_size = 32

# Упаковка датасетов в DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Информация о датасете
print("Количество изображений в train:", len(train_dataset))
print("Количество изображений в val:", len(val_dataset))
print("Список классов:", classes)
print("Количество классов:", len_classes)

Количество изображений в train: 2352
Количество изображений в val: 504
Список классов: ['acc_long_600_mg', 'advil_ultra_forte', 'akineton_2_mg', 'algoflex_forte_dolo_400_mg', 'algoflex_rapid_400_mg', 'algopyrin_500_mg', 'ambroxol_egis_30_mg', 'apranax_550_mg', 'aspirin_ultra_500_mg', 'atoris_20_mg', 'atorvastatin_teva_20_mg', 'betaloc_50_mg', 'bila_git', 'c_vitamin_teva_500_mg', 'calci_kid', 'cataflam_50_mg', 'cataflam_dolo_25_mg', 'cetirizin_10_mg', 'cold_fx', 'coldrex', 'concor_10_mg', 'concor_5_mg', 'condrosulf_800_mg', 'controloc_20_mg', 'covercard_plus_10_mg_2_5_mg_5_mg', 'coverex_4_mg', 'diclopram_75-mg_20-mg', 'dorithricin_mentol', 'dulsevia_60_mg', 'enterol_250_mg', 'favipiravir_meditop_200_mg', 'ibumax_400_mg', 'jutavit_c_vitamin', 'jutavit_cink', 'kalcium_magnezium_cink', 'kalium_r', 'koleszterin_kontroll', 'lactamed', 'lactiv_plus', 'laresin_10_mg', 'letrox_50_mikrogramm', 'lordestin_5_mg', 'merckformin_xr_1000_mg', 'meridian', 'metothyrin_10_mg', 'mezym_forte_10_000_egyseg'

In [3]:
# Этап 2. Объявление модели.

# Загрузка модели
weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights=weights)

# Замена полносвязного слоя
model.fc = nn.Linear(in_features=512, out_features=84, bias=True)

# Заморозка слоёв модели
for params in model.parameters():
    params.requires_grad = False

# Разморозка полносвязного и последнего блока свертки
for params in model.layer4.parameters():
    params.requires_grad = True

for params in model.fc.parameters():
    params.requires_grad = True

In [4]:
# Этап 3. Дообучение модели.

EPOCHS = 7

# Настройка гиперпараметров
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Обучение
training(EPOCHS, model, train_loader, val_loader, optimizer, criterion)

Эпоха 0
Эпоха: 0, батч: 19, ошибка 4.083311092853546
Эпоха: 0, батч: 39, ошибка 2.757534408569336
Эпоха: 0, батч: 59, ошибка 1.8018409430980682
В конце эпохи ошибка train 1.8018409430980682, ошибка val 1.1461925506591797
Эпоха 1
Эпоха: 1, батч: 19, ошибка 1.0447869092226028
Эпоха: 1, батч: 39, ошибка 0.830132469534874
Эпоха: 1, батч: 59, ошибка 0.7259922653436661
В конце эпохи ошибка train 0.7259922653436661, ошибка val 0.676811158657074
Эпоха 2
Эпоха: 2, батч: 19, ошибка 0.5272374480962754
Эпоха: 2, батч: 39, ошибка 0.5194489389657975
Эпоха: 2, батч: 59, ошибка 0.40263478755950927
В конце эпохи ошибка train 0.40263478755950927, ошибка val 0.32740139961242676
Эпоха 3
Эпоха: 3, батч: 19, ошибка 0.41311865001916886
Эпоха: 3, батч: 39, ошибка 0.3253709301352501
Эпоха: 3, батч: 59, ошибка 0.3046479269862175
В конце эпохи ошибка train 0.3046479269862175, ошибка val 0.2744347155094147
Эпоха 4
Эпоха: 4, батч: 19, ошибка 0.24323620684444905
Эпоха: 4, батч: 39, ошибка 0.23334115520119666
Эпоха:

In [6]:
# Этап 4. Оценка качества.

# Загрузка модели
weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights=weights)

model.fc = nn.Linear(in_features=512, out_features=84, bias=True)

model.load_state_dict(torch.load('models/meds_classifier_6.pt', weights_only=True))

labels_predicted = []
labels_true = []
 
model.eval()

with torch.no_grad():
    for data in val_loader:
        images, labels = data

        outputs = model(images)
        _, predicted = torch.max(outputs, 1) 
        labels_predicted.extend(predicted.numpy())
        labels_true.extend(labels.numpy()) 

print(classification_report(labels_true, labels_predicted, target_names=classes))


                                  precision    recall  f1-score   support

                 acc_long_600_mg       1.00      1.00      1.00         6
               advil_ultra_forte       1.00      1.00      1.00         6
                   akineton_2_mg       0.86      1.00      0.92         6
      algoflex_forte_dolo_400_mg       1.00      1.00      1.00         6
           algoflex_rapid_400_mg       1.00      1.00      1.00         6
                algopyrin_500_mg       1.00      0.67      0.80         6
             ambroxol_egis_30_mg       1.00      0.83      0.91         6
                  apranax_550_mg       1.00      1.00      1.00         6
            aspirin_ultra_500_mg       0.75      1.00      0.86         6
                    atoris_20_mg       1.00      1.00      1.00         6
         atorvastatin_teva_20_mg       0.75      1.00      0.86         6
                   betaloc_50_mg       1.00      1.00      1.00         6
                        bila_git     

1. На каких 5 классах модель ошибается чаще всего?
- Наиболее низкие показатели recall и f1-score наблюдаются у классов:
  'covercard_plus_10_mg_2_5_mg_5_mg', 'milurit_300_mg', 'theospirex_150_mg',
  'lactiv_plus', 'teva_ambrobene_30_mg'.
  Эти классы имеют самые низкие recall от 0.33 до 0.67 и f1-score до 0.5.

2. Почему модель может ошибаться на этих классах?
- Классы имеют схожие визуальные характеристики, что усложняет классификацию.
- Наименее представленные или менее характерные изображения вызывают затруднения.
- Ошибки вероятно также связаны с низким качеством изображений, что создает шумы и артефакты.

3. На каких классах модель не совершает ошибок?
- Множество классов показывают recall и precision равные 1.00, то есть модель 
  распознаёт их безошибочно. Примеры: 'acc_long_600_mg', 'advil_ultra_forte', 'aspirin_ultra_500_mg' и многие другие.

4. Почему эти классы модель распознаёт безошибочно?
- Наличие ярко выраженных уникальных признаков данных таблеток. 
- Хорошее качество и стабильность изображений.
- Достаточное количество и разнообразие обучающих примеров.

5. Как можно улучшить точность классификатора?
- Увеличить количество данных и улучшить его качество по проблемным классам.
- Использовать более глубокие архитектуры или методы ансамблирования моделей.
- Выполнить подробный ошибочный разбор и дополнительно дообучать модель на сложных примерах.

6. Как ещё можно проанализировать результаты и ошибки модели?
- Построить и визуализировать матрицу ошибок (confusion matrix) для выявления
  похожих классов, вызывающих путаницу.
- Использовать визуализацию активаций слоев (например, Grad-CAM) для интерпретации выученных признаков.
- Анализировать неправильные предсказания с визуальным просмотром изображений.
- Применять кросс-валидацию для более стабильной оценки качества.